In [ ]:
from datetime import datetime, timedelta
import os
import time

from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, collect_list, concat_ws, greatest, lit, lower, when,
    avg as _avg,
    count as _count,
    hex as _hex,
    max as _max,
    min as _min,
    round as _round,
    sum as _sum,
)
from pyspark.sql.types import (
    LongType,
)

In [ ]:
spark = SparkSession\
        .builder\
        .appName('tape-recall-history')\
        .getOrCreate()
spark

In [ ]:
# arguments
# secret path, also check if file exists
secretpath = os.environ.get('OPENSEARCH_SECRET_PATH', f'{os.getcwd()}/../workdir/secret_opensearch.txt')
with open(secretpath, 'r') as r:
    pass
# if PROD, index prefix will be `crab-*`, otherwise `crab-test-*`
PROD = os.environ.get('PROD', 'false').lower() in ('true', '1', 't')
# FROM_DATE, in strptime("%Y-%m-%d")
FROM_DATE = os.environ.get('FROM_DATE', None) 
TO_DATE = os.environ.get('TO_DATE', None)

In [ ]:
# try to import osearch from current directory, fallback to $PWD/../workdir if not found
try:
    import osearch
except ModuleNotFoundError:
    import sys
    sys.path.insert(0, f'{os.getcwd()}/../workdir')
    import osearch

In [ ]:
# notebook variables
# modify value here when run inside notebook
TODAY = "2024-09-18"
YESTERDAY = (datetime.strptime(TODAY, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d")
index_name = 'crab-test-tape-recall-history'

In [ ]:
# if cronjob, replace constant with value from env
if TO_DATE and FROM_DATE:
    TODAY = TO_DATE
    YESTERDAY = FROM_DATE
if PROD:
    index_name = f'crab-{"-".join(index_name.split("-")[2:])}'
# for osearch index pattern timestamp
timestamp_str = datetime.strptime(TODAY, "%Y-%m-%d").strftime("%s")

In [ ]:
# debug
print(TODAY)
print(YESTERDAY)
print(index_name)

In [ ]:
# Import data into spark

HDFS_RUCIO_RULES_HISTORY = f'/project/awg/cms/rucio/{TODAY}/rules_history/'

print("==============================================="
      , "RUCIO : Rules History"
      , "==============================================="
      , "File Directory:", HDFS_RUCIO_RULES_HISTORY
      , "Work Directory:", os.getcwd()
      , "==============================================="
      , "===============================================", sep='\n')

rucio_rules_history = spark.read.format('avro').load(HDFS_RUCIO_RULES_HISTORY).withColumn('ID', lower(_hex(col('ID'))))


rucio_rules_history = rucio_rules_history.select("ID", "ACCOUNT", "NAME", "STATE", "EXPIRES_AT", "UPDATED_AT", "CREATED_AT").filter(f"""ACTIVITY = 'Analysis TapeRecall'""").cache()
rucio_rules_history.createOrReplaceTempView("rules_history")

HDFS_CRAB_part = f'/project/awg/cms/crab/tasks/{TODAY}/'
print("==============================================="
      , "CRAB Table"
      , "==============================================="
      , "File Directory:", HDFS_CRAB_part
      , "Work Directory:", os.getcwd()
      , "==============================================="
      , "===============================================", sep='\n')

tasks_table = spark.read.format('avro').load(HDFS_CRAB_part)
tasks_table = tasks_table.select("TM_TASKNAME","TM_START_TIME","TM_TASK_STATUS",  'TM_TASKNAME', 'TM_START_TIME', 'TM_TASK_STATUS' , 'TM_DDM_REQID').cache()
tasks_table.createOrReplaceTempView("tasks")

In [ ]:
# Query data in daily


query = f"""\
WITH filter_t AS (
SELECT ID, ACCOUNT, NAME, STATE, EXPIRES_AT, UPDATED_AT, CREATED_AT
FROM rules_history
WHERE EXPIRES_AT < unix_timestamp("{TODAY} 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000
), 
rn_t AS (
SELECT ID, ACCOUNT, NAME, STATE, EXPIRES_AT, UPDATED_AT, CREATED_AT,
       row_number() over(partition by ID order by UPDATED_AT desc) as row_num
FROM filter_t
),
latestupdate_t AS (
SELECT * FROM rn_t 
WHERE row_num = 1
),
calc_days_t AS (
SELECT ID, ACCOUNT, NAME, STATE, EXPIRES_AT, UPDATED_AT, CREATED_AT,
       CASE 
           WHEN STATE = 'O' THEN ceil((UPDATED_AT-CREATED_AT)/86400000)  
           WHEN STATE != 'O' AND EXPIRES_AT < unix_timestamp("{TODAY} 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000 THEN ceil((EXPIRES_AT-CREATED_AT)/86400000)
           ELSE 0
       END AS DAYS
FROM latestupdate_t
),
join_t AS (
SELECT * FROM calc_days_t
LEFT JOIN tasks ON calc_days_t.ID = tasks.TM_DDM_REQID
),
window_t AS (
SELECT ID, ACCOUNT, NAME, STATE, DAYS, EXPIRES_AT, UPDATED_AT, CREATED_AT, TM_TASKNAME, TM_START_TIME, TM_TASK_STATUS, 
       row_number() OVER (PARTITION BY join_t.ID ORDER BY join_t.TM_START_TIME DESC) AS row_num
FROM join_t 
),
uniqueid_t AS (
SELECT *
FROM window_t
WHERE row_num =1
), 
final_t AS (
SELECT ID, ACCOUNT, NAME, STATE, DAYS, EXPIRES_AT, UPDATED_AT, CREATED_AT, TM_TASKNAME, IFNULL(TM_START_TIME, 0) as TM_START_TIME, TM_TASK_STATUS, 
       CREATED_AT AS timestamp,
       'crab_tape_recall_rules_history' AS type
FROM uniqueid_t
)
--- set YESTERDAY to "1970-01-01" when recompute index to get all data from begining of time
SELECT * FROM final_t
WHERE 1=1
AND EXPIRES_AT >= unix_timestamp("{YESTERDAY} 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000
AND EXPIRES_AT < unix_timestamp("{TODAY} 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000 
"""

tmpdf = spark.sql(query)
tmpdf.show()


In [ ]:
tmpdf.count()

In [ ]:
docs = tmpdf.toPandas().to_dict('records')


In [ ]:
schema = {
        "settings": {"index": {"number_of_shards": "1", "number_of_replicas": "1"}},
        "mappings": {
            "properties": {
                "ID": {"ignore_above": 2048, "type": "keyword"},
                "ACCOUNT": {"ignore_above": 2048, "type": "keyword"},
                "NAME": {"ignore_above": 2048, "type": "keyword"},
                "STATE": {"ignore_above": 2048, "type": "keyword"},
                "DAYS": {"type": "long"},
                "EXPIRES_AT": {"format": "epoch_millis", "type": "date"},
                "UPDATED_AT": {"format": "epoch_millis", "type": "date"},
                "CREATED_AT": {"format": "epoch_millis", "type": "date"},
                "TM_TASKNAME": {"ignore_above": 2048, "type": "keyword"},
                "TM_START_TIME": {"format": "epoch_millis", "type": "date"},
                "TM_TASK_STATUS": {"ignore_above": 2048, "type": "keyword"},
                "timestamp": {"format": "epoch_millis", "type": "date"},
            }
        }
    }

In [ ]:
import importlib
importlib.reload(osearch)
osearch.send_os(docs, index_name, schema, secretpath, timestamp_str)